    16_230109_100_kyungki_apt_하남
    
     
     data : /aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_하남_화성_520.xls

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_하남_화성_520.xls")
apt_df.head()

,시군명,공동주택구분,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,하남시,NaN,LH위례포엘리움,학암동,645,2021-09-14,25,6,508,Y
1,하남시,NaN,가화만사성,덕풍동,519-4,2004-11-29,10,1,72,N
2,하남시,NaN,감일 센트레빌,감이동,535,2020-03-12,25,6,589,Y
3,하남시,NaN,감일 스윗시티2단지,감일동,519,2021-05-14,25,10,684,N
4,하남시,NaN,감일스윗시티10단지,감이동,454,2020-01-15,28,9,753,N


In [5]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,하남시,LH위례포엘리움,학암동,645,2021-09-14,25,6,508,Y
1,하남시,가화만사성,덕풍동,519-4,2004-11-29,10,1,72,N
2,하남시,감일 센트레빌,감이동,535,2020-03-12,25,6,589,Y
3,하남시,감일 스윗시티2단지,감일동,519,2021-05-14,25,10,684,N
4,하남시,감일스윗시티10단지,감이동,454,2020-01-15,28,9,753,N


In [6]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [7]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,하남시,LH위례포엘리움,학암동,645,2021-09-14,25,6,508,Y,경기도 하남시 학암동 645
1,하남시,가화만사성,덕풍동,519-4,2004-11-29,10,1,72,N,경기도 하남시 덕풍동 519-4
2,하남시,감일 센트레빌,감이동,535,2020-03-12,25,6,589,Y,경기도 하남시 감이동 535
3,하남시,감일 스윗시티2단지,감일동,519,2021-05-14,25,10,684,N,경기도 하남시 감일동 519
4,하남시,감일스윗시티10단지,감이동,454,2020-01-15,28,9,753,N,경기도 하남시 감이동 454


In [8]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [9]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [10]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 하남시 학암동 645은(는) 검색에 실패했습니다
경기도 하남시 감이동 535은(는) 검색에 실패했습니다
경기도 하남시 감일동 519은(는) 검색에 실패했습니다
경기도 하남시 감이동 454은(는) 검색에 실패했습니다
경기도 하남시 감일동 534은(는) 검색에 실패했습니다
경기도 하남시 감이동 446은(는) 검색에 실패했습니다
경기도 하남시 감일동 556은(는) 검색에 실패했습니다
경기도 하남시 감이동 501은(는) 검색에 실패했습니다
경기도 하남시 감이동 448은(는) 검색에 실패했습니다
경기도 하남시 감일동 533은(는) 검색에 실패했습니다
경기도 하남시 감이동 491은(는) 검색에 실패했습니다
경기도 하남시 학암동 635은(는) 검색에 실패했습니다
경기도 하남시 학암동 652은(는) 검색에 실패했습니다
경기도 하남시 감이동 500은(는) 검색에 실패했습니다
경기도 하남시 감일동 523은(는) 검색에 실패했습니다
경기도 하남시 감일동 524은(는) 검색에 실패했습니다
경기도 하남시 감이동 530은(는) 검색에 실패했습니다
경기도 하남시 학암동 651은(는) 검색에 실패했습니다
경기도 화성시 장지동 910-311은(는) 검색에 실패했습니다
경기도 화성시 장지동 산120-19은(는) 검색에 실패했습니다
경기도 화성시 청계동 303-14은(는) 검색에 실패했습니다
경기도 화성시 동탄면 중리 627-5은(는) 검색에 실패했습니다
경기도 화성시 청계동 510-30은(는) 검색에 실패했습니다
경기도 화성시 안녕동 18050은(는) 검색에 실패했습니다
경기도 화성시 남양읍 신남리 44829은(는) 검색에 실패했습니다
경기도 화성시 청계동 49-131은(는) 검색에 실패했습니다
경기도 화성시 산척동 동탄순환대로7길 21은(는) 검색에 실패했습니다
경기도 화성시 반송동 33117은(는) 검색에 실패했습니다
경기도 화성시 영천동 108-10은(는) 검색에 실패했습니다
경기도 화성시 장지동 910-345은(는) 검색에 실패했습니다
경기도 화성시 동탄면 중리 741-3은(는

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,하남시,LH위례포엘리움,학암동,645,2021-09-14,25,6,508,Y,경기도 하남시 학암동 645,NaN,NaN
1,하남시,가화만사성,덕풍동,519-4,2004-11-29,10,1,72,N,경기도 하남시 덕풍동 519-4,973573.262038553,1948341.82748004
2,하남시,감일 센트레빌,감이동,535,2020-03-12,25,6,589,Y,경기도 하남시 감이동 535,NaN,NaN
3,하남시,감일 스윗시티2단지,감일동,519,2021-05-14,25,10,684,N,경기도 하남시 감일동 519,NaN,NaN
4,하남시,감일스윗시티10단지,감이동,454,2020-01-15,28,9,753,N,경기도 하남시 감이동 454,NaN,NaN


In [11]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [12]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,하남시,LH위례포엘리움,학암동,645,2021-09-14,25,6,508,Y,경기도 하남시 학암동 645,NaN,NaN,NaN,NaN
1,하남시,가화만사성,덕풍동,519-4,2004-11-29,10,1,72,N,경기도 하남시 덕풍동 519-4,973573.262038553,1948341.82748004,127.200886,37.534013
2,하남시,감일 센트레빌,감이동,535,2020-03-12,25,6,589,Y,경기도 하남시 감이동 535,NaN,NaN,NaN,NaN
3,하남시,감일 스윗시티2단지,감일동,519,2021-05-14,25,10,684,N,경기도 하남시 감일동 519,NaN,NaN,NaN,NaN
4,하남시,감일스윗시티10단지,감이동,454,2020-01-15,28,9,753,N,경기도 하남시 감이동 454,NaN,NaN,NaN,NaN


In [13]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [15]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [16]:
#5 seoul university 
map

In [17]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/apt_hanam.html")

In [18]:
apt_df.to_csv("apt_hanam.csv", index = False)